<a href="https://colab.research.google.com/github/charchar1245/Transformer-for-Translating-English-to-Spanish/blob/main/EnglishtoSpanishTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding data for the model -- English Spanish Translation

In [9]:
!pip uninstall -y torch torchtext torchaudio
!pip install torch==2.2.2 torchtext==0.17.2 torchaudio==2.2.2

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchtext 0.17.2
Uninstalling torchtext-0.17.2:
  Successfully uninstalled torchtext-0.17.2
Found existing installation: torchaudio 2.2.2
Uninstalling torchaudio-2.2.2:
  Successfully uninstalled torchaudio-2.2.2
  Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchtext-0.17.2-cp312-cp312-manylinux1_x86_64.whl.metadata (7.9 kB)
  Using cached torchaudio-2.2.2-cp312-cp312-manylinux1_x86_64.whl.metadata (6.4 kB)
Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl (755.5 MB)
Using cached torchtext-0.17.2-cp312-cp312-manylinux1_x86_64.whl (2.0 MB)
Using cached torchaudio-2.2.2-cp312-cp312-manylinux1_x86_64.whl (3.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.2

# Data
https://huggingface.co/datasets/okezieowen/english_to_spanish

In [1]:
import torch
import torch.nn as nn

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

ds = load_dataset("okezieowen/english_to_spanish")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

In [2]:
import pandas as pd

# To see the head of the dataset, you need to access a specific split first.
# For example, to see the first 10 elements of the 'train' split:
if 'train' in ds:
    selected_data = ds['train'].select(range(10))
    df = pd.DataFrame(selected_data)
    print(f"Displaying first 10 examples from the 'train' split:")
    display(df)
elif len(ds) > 0:
    # If 'train' split is not available, try to get the first available split
    first_split_name = list(ds.keys())[0]
    selected_data = ds[first_split_name].select(range(10))
    df = pd.DataFrame(selected_data)
    print(f"Showing first 10 examples from the '{first_split_name}' split:")
    display(df)
else:
    print("DatasetDict is empty or has no accessible splits.")

Displaying first 10 examples from the 'train' split:


,English,Spanish
0,I declare resumed the session of the European ...,Declaro reanudado el período de sesiones del P...
1,"Although, as you will have seen, the dreaded '...","Como todos han podido comprobar, el gran ""efec..."
2,You have requested a debate on this subject in...,Sus Señorías han solicitado un debate sobre el...
3,"In the meantime, I should like to observe a mi...","A la espera de que se produzca, de acuerdo con..."
4,"Please rise, then, for this minute' s silence.",Invito a todos a que nos pongamos de pie para ...
5,(The House rose and observed a minute' s silence),"(El Parlamento, de pie, guarda un minuto de si..."
6,"Madam President, on a point of order.","Señora Presidenta, una cuestión de procedimiento."
7,You will be aware from the press and televisio...,Sabrá usted por la prensa y la televisión que ...
8,One of the people assassinated very recently i...,Una de las personas que recientemente han ases...
9,"Would it be appropriate for you, Madam Preside...","¿Sería apropiado que usted, Señora Presidenta,..."


In [3]:
# Ensure spaCy English model is loaded if not already
# If you encounter an error, uncomment and run the following line:
# !python -m spacy download en_core_web_sm
# spacy_en = spacy.load('en_core_web_sm')

# Define the tokenization function (if not already defined)
def tokenize_en(text):
    """Tokenizes English text using a spaCy en_core_web_sm model and returns a list of tokens."""
    # Check if spacy_en is defined globally, otherwise load it
    if 'spacy_en' not in globals():
        global spacy_en
        import spacy
        spacy_en = spacy.load('en_core_web_sm')
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Apply tokenization to the 'English' column of the 'train' split
# Filter out None values before tokenizing
tokenized_english_sentences = [tokenize_en(sentence) for sentence in ds['train']['English'] if sentence is not None]

# Display the first 5 tokenized English sentences
print("First 5 tokenized English sentences:")
for i, tokens in enumerate(tokenized_english_sentences[:5]):
    print(tokens)

First 5 tokenized English sentences:
['I', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'European', 'Parliament', 'adjourned', 'on', 'Friday', '17', 'December', '1999', ',', 'and', 'I', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period', '.']
['Although', ',', 'as', 'you', 'will', 'have', 'seen', ',', 'the', 'dreaded', "'", 'millennium', 'bug', "'", 'failed', 'to', 'materialise', ',', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful', '.']
['You', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days', ',', 'during', 'this', 'part', '-', 'session', '.']
['In', 'the', 'meantime', ',', 'I', 'should', 'like', 'to', 'observe', 'a', 'minute', "'", 's', 'silence', ',', 'as', 'a', 'number', 'of', 'Me

First, let's build a vocabulary from our `tokenized_english_sentences`. This vocabulary will map each unique word to a unique integer ID.

In [4]:
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
import torch.nn as nn
import math

# Flatten the list of tokenized sentences to get all tokens
all_tokens = [token for sentence in tokenized_english_sentences for token in sentence]

# Build vocabulary with special tokens
UNK_IDX = 0 # Unknown token index
PAD_IDX = 1 # Padding token index

vocab = build_vocab_from_iterator(
    [sentence for sentence in tokenized_english_sentences], # Pass an iterator of token lists
    min_freq=1,
    specials=['<unk>', '<pad>', '<bos>', '<eos>'],
    special_first=True
)
vocab.set_default_index(UNK_IDX)

print(f"Vocabulary size: {len(vocab)}")
print(f"Example mapping: 'hello' -> {vocab(['hello'])[0]}, '<pad>' -> {vocab(['<pad>'])[0]}")

Vocabulary size: 37310
Example mapping: 'hello' -> 33348, '<pad>' -> 1


Next, we'll numericalize the tokenized sentences (convert tokens to their vocabulary IDs) and then pad them to a consistent length. We'll choose a maximum sequence length, and either pad shorter sentences or truncate longer ones.

In [5]:
MAX_SEQ_LEN = 100 # Define a maximum sequence length

def numericalize_and_pad(sentences, vocab, max_seq_len, pad_idx=PAD_IDX):
    numericalized_sentences = []
    for sentence in sentences:
        # Add <bos> and <eos> tokens
        indexed_sentence = [vocab['<bos>']] + vocab(sentence) + [vocab['<eos>']]

        # Pad or truncate
        if len(indexed_sentence) < max_seq_len:
            padded_sentence = indexed_sentence + [pad_idx] * (max_seq_len - len(indexed_sentence))
        else:
            padded_sentence = indexed_sentence[:max_seq_len]

        numericalized_sentences.append(padded_sentence)
    return torch.tensor(numericalized_sentences)

# Numericalize and pad the English sentences
input_sequences_numerical = numericalize_and_pad(tokenized_english_sentences, vocab, MAX_SEQ_LEN, PAD_IDX)

print(f"Shape of numerical input sequences: {input_sequences_numerical.shape}")
print("First numerical input sequence:", input_sequences_numerical[0])

Shape of numerical input sequences: torch.Size([149829, 100])
First numerical input sequence: tensor([    2,    14,  2756,  2673,     4,  1028,     7,     4,    30,    50,
         4451,    16,  2462,  2104,  1083,   587,     5,     9,    14,    36,
           62,   367,   261,     8,   307,    52,    12,  1472,    96,   166,
           10,     4,   198,    13,    52,  4087,    12,  6453, 18740,   537,
            6,     3,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])


Now, we define an embedding layer to convert these numerical IDs into dense vector representations. After that, we'll add positional encodings, which are crucial for transformers to understand the order of words in a sequence.

In [6]:
from torch.utils.data import DataLoader, TensorDataset

EMBEDDING_DIM = 256 # Dimension of word embeddings

# Create the embedding layer
embedding_layer = nn.Embedding(len(vocab), EMBEDDING_DIM, padding_idx=PAD_IDX)

BATCH_SIZE = 32 # Batch size
dataset = TensorDataset(input_sequences_numerical)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Pass the numerical sequences through the embedding layer
# input_embeddings = embedding_layer(input_sequences_numerical)
for batch_idx, (batch,) in enumerate(dataloader):
    # batch: (batch_size, seq_len)

    input_embeddings = embedding_layer(batch)
    # input_embeddings: (batch_size, seq_len, embedding_dim)

    # 🔹 later: positional encoding, attention, loss, backprop

    if batch_idx == 0:
        print(input_embeddings.shape)

torch.Size([32, 100, 256])


In [7]:

print(f"Shape of input embeddings: {input_embeddings.shape}") # (batch_size, seq_len, embedding_dim)

# Define Positional Encoding module
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        # self.pe: (1, max_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return x

# Instantiate and apply positional encoding
positional_encoder = PositionalEncoding(EMBEDDING_DIM, MAX_SEQ_LEN)
input_embeddings_with_pos = positional_encoder(input_embeddings)

print(f"Shape of input embeddings with positional encoding: {input_embeddings_with_pos.shape}")

Shape of input embeddings: torch.Size([5, 100, 256])
Shape of input embeddings with positional encoding: torch.Size([5, 100, 256])


First, let's define the `MultiHeadSelfAttention` module. This module allows the model to jointly attend to information from different representation subspaces at different positions.

In [8]:
import torch.nn as nn
import torch

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        assert (
            self.head_dim * num_heads == embed_dim
        ), "embed_dim must be divisible by num_heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(num_heads * self.head_dim, embed_dim)

    def forward(self, value, key, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = value.shape[1], key.shape[1], query.shape[1]

        # Split the embedding into self.num_heads different pieces
        value = value.reshape(N, value_len, self.num_heads, self.head_dim)
        key = key.reshape(N, key_len, self.num_heads, self.head_dim)
        query = query.reshape(N, query_len, self.num_heads, self.head_dim)

        values = self.values(value)
        keys = self.keys(key)
        queries = self.queries(query)

        # Einsum does matrix multiplication for query * key.T
        # with shape (N, heads, query_len, head_dim) * (N, heads, head_dim, key_len) -> (N, heads, query_len, key_len)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_dim ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.num_heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Example Usage:
# N = 2 # Batch size
# seq_len = 10 # Sequence length
# embed_dim = 256 # Embedding dimension
# num_heads = 8 # Number of attention heads

# value = torch.randn(N, seq_len, embed_dim)
# key = torch.randn(N, seq_len, embed_dim)
# query = torch.randn(N, seq_len, embed_dim)
# mask = torch.ones(N, 1, seq_len, seq_len) # Example mask (no masking for simplicity)

# attention_block = MultiHeadSelfAttention(embed_dim, num_heads)
# output = attention_block(value, key, query, mask)
# print(f"Output shape of MultiHeadSelfAttention: {output.shape}")

Next, we define a simple `FeedForwardBlock`, which is applied to each position separately and identically. This typically consists of two linear transformations with a ReLU activation in between.

In [9]:
class FeedForwardBlock(nn.Module):
    def __init__(self, embed_dim, forward_expansion):
        super(FeedForwardBlock, self).__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * forward_expansion),
            nn.ReLU(),
            nn.Linear(embed_dim * forward_expansion, embed_dim),
        )

    def forward(self, x):
        return self.feed_forward(x)

# Example Usage:
# N = 2 # Batch size
# seq_len = 10 # Sequence length
# embed_dim = 256 # Embedding dimension
# forward_expansion = 4

# x = torch.randn(N, seq_len, embed_dim)

# ff_block = FeedForwardBlock(embed_dim, forward_expansion)
# output = ff_block(x)
# print(f"Output shape of FeedForwardBlock: {output.shape}")

Finally, we combine these two components into an `EncoderBlock`. This block integrates multi-head self-attention, a feed-forward network, residual connections, and layer normalization, forming a fundamental building block of a Transformer encoder.

In [10]:
class EncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, forward_expansion, dropout):
        super(EncoderBlock, self).__init__()
        self.attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.feed_forward = FeedForwardBlock(embed_dim, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        # Multi-head self-attention with residual connection and layer normalization
        attention = self.attention(x, x, x, mask)
        x = self.dropout(self.norm1(attention + x))

        # Feed-forward network with residual connection and layer normalization
        feed_forward = self.feed_forward(x)
        out = self.dropout(self.norm2(feed_forward + x))
        return out

# Example Usage:
# N = 2 # Batch size
# seq_len = 10 # Sequence length
# embed_dim = 256 # Embedding dimension
# num_heads = 8 # Number of attention heads
# forward_expansion = 4 # Expansion factor for feed-forward layer
# dropout = 0.1 # Dropout rate

# x = torch.randn(N, seq_len, embed_dim)
# mask = torch.ones(N, 1, seq_len, seq_len) # Example mask

# encoder_block = EncoderBlock(embed_dim, num_heads, forward_expansion, dropout)
# output = encoder_block(x, mask)
# print(f"Output shape of EncoderBlock: {output.shape}")

Let's test the `EncoderBlock` with the `input_embeddings_with_pos` we generated earlier. Note that `input_embeddings_with_pos` had a batch size of 5 from the previous `dataloader` iteration, not the full `BATCH_SIZE` of 32.

In [11]:
from torch.utils.data import DataLoader, TensorDataset

# Redefine the batch size to match what was in input_embeddings_with_pos
# The previous run only processed 5 items in the last batch if the dataset size was not divisible by BATCH_SIZE
# Or, to be safe, let's just make a dummy input with the current batch size used in input_embeddings_with_pos
current_batch_size = input_embeddings_with_pos.shape[0]

# Assuming input_embeddings_with_pos comes from the previous run with a certain batch size
# We will use this size for our dummy input to test the EncoderBlock

# Set parameters for the EncoderBlock
EMBEDDING_DIM = input_embeddings_with_pos.shape[2] # Use the actual embedding dimension
NUM_HEADS = 8  # A common choice, adjust as needed
FORWARD_EXPANSION = 4 # A common choice, adjust as needed
DROPOUT = 0.1 # A common choice, adjust as needed

# Create a dummy mask for testing. For an encoder, this is typically all ones (no masking within sequence).
# Or it can be used for padding mask, where padded tokens are masked out.
# Here we assume no padding mask for simplicity in this test.
mask = torch.ones(current_batch_size, 1, MAX_SEQ_LEN, MAX_SEQ_LEN)

# Instantiate the EncoderBlock
encoder_block = EncoderBlock(EMBEDDING_DIM, NUM_HEADS, FORWARD_EXPANSION, DROPOUT)

# Pass the input embeddings with positional encoding through the EncoderBlock
output_encoder_block = encoder_block(input_embeddings_with_pos, mask)

print(f"Shape of output after EncoderBlock: {output_encoder_block.shape}")

Shape of output after EncoderBlock: torch.Size([5, 100, 256])


# Task
Tokenize the Spanish sentences in the dataset using spaCy's `es_core_news_sm` model, then build a Spanish vocabulary, and numericalize and pad the tokenized Spanish sentences.

## Tokenize Spanish Sentences

### Subtask:
Define a tokenization function for Spanish text using spaCy's es_core_news_sm model and apply it to the Spanish column of the dataset.


**Reasoning**:
I need to define a tokenization function for Spanish text using spaCy and apply it to the 'Spanish' column of the dataset. This involves importing spaCy, defining the `tokenize_es` function to handle model loading and tokenization, and then applying this function to the relevant dataset column while handling potential `None` values.



In [12]:
import spacy

# Ensure spaCy Spanish model is loaded if not already
# If you encounter an error, uncomment and run the following line:
# !python -m spacy download es_core_news_sm

# Define the tokenization function for Spanish
def tokenize_es(text):
    """Tokenizes Spanish text using a spaCy es_core_news_sm model and returns a list of tokens."""
    # Check if spacy_es is defined globally, otherwise load it
    if 'spacy_es' not in globals():
        global spacy_es
        spacy_es = spacy.load('es_core_news_sm')
    return [tok.text for tok in spacy_es.tokenizer(text)]

# Apply tokenization to the 'Spanish' column of the 'train' split
# Filter out None values before tokenizing
tokenized_spanish_sentences = [tokenize_es(sentence) for sentence in ds['train']['Spanish'] if sentence is not None]

# Display the first 5 tokenized Spanish sentences
print("First 5 tokenized Spanish sentences:")
for i, tokens in enumerate(tokenized_spanish_sentences[:5]):
    print(tokens)


OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

**Reasoning**:
The previous cell failed because the `es_core_news_sm` spaCy model was not found. I need to install it before attempting to load it.



In [13]:
!python -m spacy download es_core_news_sm

import spacy

# Define the tokenization function for Spanish
def tokenize_es(text):
    """Tokenizes Spanish text using a spaCy es_core_news_sm model and returns a list of tokens."""
    # Check if spacy_es is defined globally, otherwise load it
    if 'spacy_es' not in globals():
        global spacy_es
        spacy_es = spacy.load('es_core_news_sm')
    return [tok.text for tok in spacy_es.tokenizer(text)]

# Apply tokenization to the 'Spanish' column of the 'train' split
# Filter out None values before tokenizing
tokenized_spanish_sentences = [tokenize_es(sentence) for sentence in ds['train']['Spanish'] if sentence is not None]

# Display the first 5 tokenized Spanish sentences
print("First 5 tokenized Spanish sentences:")
for i, tokens in enumerate(tokenized_spanish_sentences[:5]):
    print(tokens)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.12/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.12/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.12/dist-packages/spacy/compat.py", line 5, in <module>
    from thinc.

**Reasoning**:
The previous step successfully tokenized the Spanish sentences. Now I need to build a vocabulary from these tokenized Spanish sentences, similar to how the English vocabulary was built. This is the next logical step in preparing the Spanish data for the model.



In [14]:
from torchtext.vocab import build_vocab_from_iterator

# Build vocabulary for Spanish with special tokens
# UNK_IDX and PAD_IDX are already defined globally from the English vocabulary creation
# UNK_IDX = 0
# PAD_IDX = 1

vocab_es = build_vocab_from_iterator(
    [sentence for sentence in tokenized_spanish_sentences], # Pass an iterator of token lists
    min_freq=1,
    specials=['<unk>', '<pad>', '<bos>', '<eos>'],
    special_first=True
)
vocab_es.set_default_index(UNK_IDX)

print(f"Spanish Vocabulary size: {len(vocab_es)}")
print(f"Example mapping: 'hola' -> {vocab_es(['hola'])[0]}, '<pad>' -> {vocab_es(['<pad>'])[0]}")

Spanish Vocabulary size: 66909
Example mapping: 'hola' -> 0, '<pad>' -> 1


**Reasoning**:
Now that the Spanish vocabulary has been built, the next step is to numericalize the tokenized Spanish sentences (convert tokens to their vocabulary IDs) and then pad them to a consistent length, similar to how the English sentences were processed. This will prepare the Spanish data for model input.



In [15]:
MAX_SEQ_LEN = 100 # Ensure MAX_SEQ_LEN is defined or accessible

# Numericalize and pad the Spanish sentences
output_sequences_numerical = numericalize_and_pad(tokenized_spanish_sentences, vocab_es, MAX_SEQ_LEN, PAD_IDX)

print(f"Shape of numerical output sequences: {output_sequences_numerical.shape}")
print("First numerical output sequence:", output_sequences_numerical[0])

Shape of numerical output sequences: torch.Size([149559, 100])
First numerical output sequence: tensor([   2, 5649, 8464,   10,  572,    4, 1219,   16,   37,   82,    5, 5005,
          10, 2013, 2062,    4, 1082,  280,    5,   11, 9664,   12, 2202,  273,
          75,  607,    4,    7,  830,  491,  434, 1362, 5570,    8,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])


## Define Decoder Block

### Subtask:
Define the `DecoderBlock` module. This block will incorporate masked multi-head self-attention (for the target sequence), multi-head attention over the encoder's output, a feed-forward network, residual connections, and layer normalization, following the standard Transformer architecture.


**Reasoning**:
I need to define the `DecoderBlock` class as instructed, incorporating masked multi-head self-attention, multi-head attention over the encoder's output, a feed-forward network, residual connections, layer normalization, and dropout layers, following the provided structure.



In [16]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        # Masked Multi-Head Self-Attention for the target sequence
        self.self_attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)

        # Multi-Head Attention over the encoder's output
        self.encoder_attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout2 = nn.Dropout(dropout)

        # Feed-forward network
        self.feed_forward = FeedForwardBlock(embed_dim, forward_expansion)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_output, trg_mask, src_mask):
        # Masked Multi-Head Self-Attention
        # Query, Key, Value are all from the target sequence (x)
        self_attention = self.self_attention(x, x, x, trg_mask)
        # Add residual connection, apply dropout and layer normalization
        x = self.dropout1(self.norm1(self_attention + x))

        # Multi-Head Encoder-Decoder Attention
        # Query from target sequence (x), Key and Value from encoder output (enc_output)
        encoder_attention = self.encoder_attention(enc_output, enc_output, x, src_mask)
        # Add residual connection, apply dropout and layer normalization
        x = self.dropout2(self.norm2(encoder_attention + x))

        # Feed-forward network
        feed_forward = self.feed_forward(x)
        # Add residual connection, apply dropout and layer normalization
        out = self.dropout3(self.norm3(feed_forward + x))

        return out

print("DecoderBlock class defined.")

DecoderBlock class defined.


**Reasoning**:
The `DecoderBlock` class has been defined. Now, I need to test its functionality by creating a dummy input, `enc_output`, and the necessary masks, then instantiating the `DecoderBlock` and passing these through its `forward` method. This will verify that the block can process inputs and produce an output with the expected shape.



In [17]:
import torch

# Assuming embed_dim, num_heads, forward_expansion, dropout are already defined globally
# EMBEDDING_DIM = 256
# NUM_HEADS = 8
# FORWARD_EXPANSION = 4
# DROPOUT = 0.1

# Example Usage:
N = 2 # Batch size
seq_len_trg = 15 # Target sequence length (for x)
seq_len_src = MAX_SEQ_LEN # Source sequence length (for enc_output, from MAX_SEQ_LEN)

# Dummy input for the decoder (target embeddings)
x = torch.randn(N, seq_len_trg, EMBEDDING_DIM)

# Dummy encoder output
enc_output = torch.randn(N, seq_len_src, EMBEDDING_DIM)

# Dummy target mask (for masked self-attention in decoder)
# This mask typically ensures that a token cannot attend to future tokens.
# For simplicity, let's use a causal mask. (seq_len_trg, seq_len_trg)
trg_mask = torch.tril(torch.ones(seq_len_trg, seq_len_trg)).unsqueeze(0).unsqueeze(0).expand(N, NUM_HEADS, seq_len_trg, seq_len_trg)

# Dummy source mask (for encoder-decoder attention)
# This mask typically handles padding in the encoder output. Here, assuming no padding mask needed.
# For simplicity, we can use an all-ones mask if no actual padding mask is being generated.
src_mask = torch.ones(N, 1, 1, seq_len_src) # N, 1, 1, src_seq_len for broadcasting

# Instantiate the DecoderBlock
decoder_block = DecoderBlock(EMBEDDING_DIM, NUM_HEADS, FORWARD_EXPANSION, DROPOUT)

# Pass the inputs through the DecoderBlock
output_decoder_block = decoder_block(x, enc_output, trg_mask, src_mask)

print(f"Shape of output after DecoderBlock: {output_decoder_block.shape}")

Shape of output after DecoderBlock: torch.Size([2, 15, 256])


## Final Task

### Subtask:
Summarize the successful implementation and testing of the Decoder Block and prepare for integrating it into a full Transformer model.


## Summary:

### Data Analysis Key Findings

*   The Spanish sentence tokenization was successfully performed using spaCy's `es_core_news_sm` model, processing 149,559 sentences.
*   A Spanish vocabulary was constructed, comprising 66,909 unique tokens, including special tokens like `<unk>`, `<pad>`, `<bos>`, and `<eos>`.
*   The tokenized Spanish sentences were numericalized and padded to a uniform length of 100, resulting in a tensor of shape `(149559, 100)`.
*   The `DecoderBlock` module was successfully defined, incorporating key Transformer components: masked multi-head self-attention, encoder-decoder attention, a feed-forward network, residual connections, and layer normalization.
*   The functionality of the `DecoderBlock` was validated with dummy inputs, producing an output of shape `(2, 15, 256)`, which aligns with expected dimensions for a batch size of 2, a target sequence length of 15, and an embedding dimension of 256.

### Insights or Next Steps

*   The successfully tokenized, vocabulary-built, numericalized, and padded Spanish dataset is now ready for use as target sequences in a sequence-to-sequence model.
*   The validated `DecoderBlock` can now be integrated into a complete Transformer Decoder, and subsequently into a full Transformer model for machine translation or other sequence generation tasks.
